# SQL Overview
SQL (Structured Query Language) is a domain-specific language used in programming and designed for managing data held in a relational database management system, or for stream processing in a relational data stream management system. 

**Important Note:** I am using a MySQL database for all examples in this notebook. SQL queries differ slightly from database to database, I detail other db examples in the databases notebook inside of anaconda-projects/data-science/fundamentals

1. Basic SQL Statements
2. Creating & Editing Tables In SQL
3. SubQueries
3. khkjh 
4. kjh kjh
5. hkjhjh

These are good refresher [Tutorial](https://www.youtube.com/watch?v=a9W7OpS4LfI&list=PLyuRouwmQCjlXvBkTfGeDTq79r9_GoMt9&ab_channel=SteveGriffith-Prof3ssorSt3v3) videos

In [25]:
import requests
import numpy as np
import pandas as pd
import mysql.connector 

conn = mysql.connector.connect(host='localhost',
                               user='root',
                               password='InfoDump21!')

db1 = 'netflix'
db2 = 'band'
limit = 5

print(conn)

cur = conn.cursor()
cur.execute(f'CREATE DATABASE IF NOT EXISTS {db1};')

print(f'\nConnection to `{db1}` Database Created Successfully')

cur.close()
conn.close()
print(f'\n`{db1}` Database Connection Closed.\n')


Connection to `netflix` Database Created Successfully

`netflix` Database Connection Closed.



In [26]:
def run_query(query, db):
    try:
        conn = mysql.connector.connect(host='localhost',
                                       user='root',
                                       password='InfoDump21!',
                                       database=db)
        return pd.read_sql_query(query, conn)
    except:
        print(f'Error: could not connect to {db} database.')

def run_update(query, db, mode):
    conn = mysql.connector.connect(host='localhost',
                                   user='root',
                                   password='InfoDump21!',
                                   database=db)
    
    # Create cursor and use it to perform queries
    cur = conn.cursor()
    cur.execute(query)
    
    if (mode == 'insert') or (mode == 'delete') or (mode == 'update') or (mode == 'replace'):
        conn.commit()
    
    cur.close()
    conn.close()

## 1. Basic SQL Statements
All SQL statements are run using SQL keyword which by convention are usually all in caps, however this is not necessary for them to work.

**USE** - Selects the database you want to use, in the examples here I have already selected this inside of the run_query function

**DESCRIBE** - This keyword give the field characteristics of a database tables

In [203]:
run_query('DESCRIBE movies', db1)

,Field,Type,Null,Key,Default,Extra
0,id,b'int',YES,,None,
1,title,b'varchar(255)',YES,,None,
2,director,b'varchar(100)',YES,,None,
3,release_date,b'int',YES,,None,
4,runtime,b'int',YES,,None,
5,rating,b'varchar(15)',YES,,None,
6,category,b'varchar(50)',YES,,None,


**SELECT** - Used when wanting to select or pull information from a database table
Select statements always have a **FROM** keyword as well to signify which table to pull from

**LIMIT** - Used to limit the number of rows returned. A single value limits to only that number (i.e. LIMIT 10 limits to 10 rows returned) whereas (LIMIT 5, 10 starts at the 6th returned value and returns the next 10 rows)

In [204]:
run_query(f'SELECT * FROM movies LIMIT {limit}', db1)

,id,title,director,release_date,runtime,rating,category
0,81145628,Norm of the North: King Sized Adventure,Richard Finn,2019,90,TV-PG,Children & Family Movies
1,80125979,#realityhigh,Fernando Lebrija,2017,99,TV-14,Comedies
2,70304989,Automata,Gabe Ibanez,2014,110,R,International Movies
3,80164077,Fabrizio Copano: Solo pienso en mi,Rodrigo Toro,2017,60,TV-MA,Stand-Up Comedy
4,70304990,Good People,Henrik Ruben Genz,2014,90,R,Action & Adventure


**WHERE** - is used to conditinally filter by fields (column values)

In [205]:
query = f"""
SELECT *
FROM movies
WHERE release_date = 2019
LIMIT {5}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,81145628,Norm of the North: King Sized Adventure,Richard Finn,2019,90,TV-PG,Children & Family Movies
1,81154455,Article 15,Anubhav Sinha,2019,125,TV-MA,Dramas
2,81132437,Kill Me If You Dare,Senol Sonmez,2019,100,TV-14,Comedies
3,81078908,The World We Make,Brian Baugh,2019,108,PG,Dramas
4,81155784,Watchman,A. L. Vijay,2019,93,TV-14,Comedies


**LIKE** - Allows the use of wildcard symbols (think regex) to match specific parts of a string, this link has some of the more common exampels of LIKE's usage: [LIKE examples](https://www.tutorialspoint.com/sql/sql-like-clause.htm)

In [206]:
# This query finds all movies with the word 'star' in it
# note that case-sensitivity is determined on table creation

query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,70267838,20 Feet From Stardom,Morgan Neville,2013,91,PG-13,Documentaries
1,80092857,Pup Star,Robert Vince,2016,92,G,Children & Family Movies
2,80189954,Pup Star: Better 2Gether,Robert Vince,2017,93,PG,Children & Family Movies
3,80998890,Upstarts,Udai Singh Pawar,2019,112,TV-14,Comedies
4,80112370,Barbie Star Light Adventure,Andrew Tan,2016,79,TV-Y,Children & Family Movies


**AND/OR** - two conditional statements allowing for more required conditions within queries. AND means that all options must exist to be true, OR means either can exist to be true

In [207]:
# This query searches for movies with star in the title with length greater than 90 minutes
query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
AND (runtime > 100 OR category LIKE '%Comedies%')
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,80998890,Upstarts,Udai Singh Pawar,2019,112,TV-14,Comedies
1,28631236,Superstar,Bruce McCulloch,1999,82,PG-13,Comedies
2,81010865,Starting Over Again,Olivia M. Lamasan,2014,128,TV-14,Dramas
3,80245408,Secret Superstar,Advait Chandan,2017,150,TV-14,Dramas
4,70117293,The Men Who Stare at Goats,Grant Heslov,2009,94,R,Comedies


**ORDER BY** - is used to sort the data by field names (numerically or alphabetically) in either ascending (ASC) or descending (DESC) orders

In [208]:
# This query searches for movies with star in the title with length greater than 90 minutes
query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
AND 100
ORDER BY runtime DESC
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,80192018,Star Wars: Episode VIII: The Last Jedi,Rian Johnson,2017,152,PG-13,Action & Adventure
1,80245408,Secret Superstar,Advait Chandan,2017,150,TV-14,Dramas
2,81046962,Solo: A Star Wars Story (Spanish Version),Ron Howard,2018,135,PG-13,Action & Adventure
3,80220814,Solo: A Star Wars Story,Ron Howard,2018,135,PG-13,Action & Adventure
4,81010865,Starting Over Again,Olivia M. Lamasan,2014,128,TV-14,Dramas


**Aliasing using AS** - the AS command is used to rename a column or table with an alias which only exists for the duration of the query. This is used often when field names are confusion or really long. 

In [209]:
# In this example the field name 'director' or 'release_date' is changed to 'dir' and 'date'
# the movies table is also changed to 'm'
# note that table aliases don't work within their respective queries with WHERE etc.,
# below WHERE release_date (or m.release_date) works, but date or m.date would not

query = f"""
SELECT m.title, m.director AS dir, m.release_date AS date
FROM `movies` AS m
WHERE release_date = 2019
LIMIT {limit}
"""
run_query(query, db1)

,title,dir,date
0,Norm of the North: King Sized Adventure,Richard Finn,2019
1,Article 15,Anubhav Sinha,2019
2,Kill Me If You Dare,Senol Sonmez,2019
3,The World We Make,Brian Baugh,2019
4,Watchman,A. L. Vijay,2019


**DISTINCT** - is used to return the unique values in a column

In [210]:
# This query pulls all possible release dates for the movies 

query = f"""
SELECT DISTINCT release_date
FROM movies
ORDER BY release_date DESC
LIMIT {limit}
"""
run_query(query, db1)

,release_date
0,2020
1,2019
2,2018
3,2017
4,2016


## 2. Creating & Editing Tables In SQL
There are a number of different keywords used for creating, removing, and changing table content including:

**CREATE** - allows tables to be created from existing tables or from scratch

**DROP** - keyword used for deleting databases, tables, and columns from tables

**ALTER** - keyword used for altering table field (column) types and characterstics (for the table itself, not the values)

**INSERT** - inserts BRAND NEW records into a table (i.e. new row)

**UPDATE** - modifies exisiting table field values

**DELETE** - deletes existing table records (i.e. entire rows)

**REPLACE** - works exactly like INSERT, except that if an old row in the table has the same value as a new row for a PRIMARY KEY or a UNIQUE index, the old row is deleted before the new row is inserted. NOTE that this is not the same as the REPLACE() function which replaces all occurences of a substring in a string. 

### CREATE a table from an existing table

In [212]:
# Create a table called `comedies` to store all the comedies in the movies table

query = f"""
CREATE TABLE IF NOT EXISTS comedies AS
SELECT title, director, release_date AS date, runtime, rating FROM `movies` 
WHERE category = 'Comedies';
"""

test1 = f"""
SELECT COUNT(*) FROM comedies
LIMIT {limit};
"""

test2 = f"""
SELECT * FROM comedies
LIMIT {limit};
"""

run_update(query, db1, 'create')

print('Number of Rows Returned:')
print(run_query(test1, db1))
run_query(test2, db1)

Number of Rows Returned:
   COUNT(*)
0       787


,title,director,date,runtime,rating
0,#realityhigh,Fernando Lebrija,2017,99,TV-14
1,Manhattan Romance,Tom O'Brien,2014,98,TV-14
2,Sierra Burgess Is A Loser,Ian Samuels,2018,106,PG-13
3,Care of Kancharapalem,Maha Venkatesh,2018,142,TV-14
4,Ee Nagaraniki Emaindi,Tharun Bhascker,2018,133,TV-14


### CREATE  a table from scratch

In [47]:
# Here I create a table called memebers for a new database called band

create_table = f"""
CREATE TABLE IF NOT EXISTS members(
member_id INT NOT NULL AUTO_INCREMENT,
fname VARCHAR(50) NOT NULL,
lname VARCHAR(50) NOT NULL,
instrument VARCHAR(50) NOT NULL,
PRIMARY KEY (member_id)
)
"""

run_update(create_table, db2, 'create')

run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,


### ALTER a table to add a new column
I forgot to add the band name field, so I use the ALTER keyword to do so

In [48]:
# Add the field band_name to the members table

create_table = f"""
ALTER TABLE members
ADD COLUMN band_name VARCHAR(75)
"""
run_update(create_table, db2, 'alter')
run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,
4,band_name,b'varchar(75)',YES,,None,


### MODIFY altered table 
I forgot to make the band_names field NOT NULL so below I modify the column to rectify the situation, note that COLUMN is not needed after MODIFY or ADD from above, but it makes the query more understandable

In [49]:
# Add the field band_name to the members table

create_table = f"""
ALTER TABLE members
MODIFY COLUMN band_name VARCHAR(75) NOT NULL
"""
run_update(create_table, db2, 'alter')
run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,
4,band_name,b'varchar(75)',NO,,None,


### INSERT records into a table

In [50]:
# Here I insert 4 band members into the table

insert_data = f"""
INSERT INTO members (fname, lname, instrument, band_name)
VALUES
  ('Paul', 'Bullard', 'Guitar', 'Roof Diver'),
  ('Cameron', 'McClain', 'Bass', 'Roof Diver'),
  ('Eric', 'Elliot', 'Guitar', 'Roof Diver'),
  ('Rodney', 'Walton', 'Drums', 'Roof Diver');
"""

query = """
SELECT * FROM members;
"""
run_update(insert_data, db2, 'insert')

run_query(query, db2)

,member_id,fname,lname,instrument,band_name
0,1,Paul,Bullard,Guitar,Roof Diver
1,2,Cameron,McClain,Bass,Roof Diver
2,3,Eric,Elliot,Guitar,Roof Diver
3,4,Rodney,Walton,Drums,Roof Diver


### UPDATING Table Values
The UPDATE keyword along with SET is used to update existing values in a table
In the example below I create a new column for vocals and make the default = 'no'.
Then I update this to yes for memeber who sing

In [51]:
# Create a new column for vocals
add_column = f"""
ALTER TABLE members
ADD COLUMN vocals VARCHAR(5) DEFAULT 'no'
"""

run_update(add_column, db2, 'alter')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,no
1,2,Cameron,McClain,Bass,Roof Diver,no
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no


In [52]:
# Update the vocals column to 'yes' for singing members
update_vox = f"""
UPDATE members
SET vocals = 'yes'
WHERE member_id = 1 OR member_id = 2;
"""

run_update(update_vox, db2, 'update')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no


### DELETE a record
This is as simple as it sounds, but I add another few rows to the dataset beforehand

In [53]:
# Adding in a few more rows (most with incorrect data for later examples)
insert_data = f"""
INSERT INTO members (fname, lname, instrument, band_name, vocals)
VALUES
  ('John', 'Lennon', 'Guitar', 'Rolling Stones', 'yes'),
  ('Paul', 'McCartney', 'Drums', 'The Beatles', 'no'),
  ('Ringo', 'Star', 'Drums', 'The Beatles', 'yes'),
  ('George', 'Hamilton', 'Guitar', 'The Beatles', 'yes'),
  ('Jack', 'Spratt', 'Triangle', 'Yo Yo Yo', 'no');
"""

query = """
SELECT * FROM members;
"""
run_update(insert_data, db2, 'insert')

run_query(query, db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no
4,5,John,Lennon,Guitar,Rolling Stones,yes
5,6,Paul,McCartney,Drums,The Beatles,no
6,7,Ringo,Star,Drums,The Beatles,yes
7,8,George,Hamilton,Guitar,The Beatles,yes
8,9,Jack,Spratt,Triangle,Yo Yo Yo,no


In [54]:
# Delete the last row because it it's not valid 
update_vox = f"""
DELETE FROM members
WHERE member_id = 9;
"""

run_update(update_vox, db2, 'delete')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no
4,5,John,Lennon,Guitar,Rolling Stones,yes
5,6,Paul,McCartney,Drums,The Beatles,no
6,7,Ringo,Star,Drums,The Beatles,yes
7,8,George,Hamilton,Guitar,The Beatles,yes


## 3. Joins
Joins are used to combine tables in various ways based off of a common variables (usually primary and/or foreign keys). There are 4 main types of Joins:

* **Inner Join** - Combines tables where only common variable matches
* **Self Join** - An inner join that joins values from the same table (usually the second table is cretead as an alias), an example of this would be having a table with an employee id and a boss id, then combining the employee with the boss
* **Left Join** - Combines everything from left table with matching common variable from right
* **Right Join** - Combines everything from right table with matching common variable from left
* **Outer Joins** - Combines everything from both tables regardless of match

Left/Right/and Outer joins can also be adjusted slightly for different results froma above. See the chart below for a visual on all types of joins:

<img src='data/images/sql1.png' width='700' >


## Setting up the band database for join examples
Below I create a new table inside the band database called locations and also update the members table so that I can better explain joins. This is super contrived, but it should get the point across

In [55]:
# Below I create a new table called location for the bands so that I can show join examples
create_table = f"""
CREATE TABLE IF NOT EXISTS locations(
location_id INT NOT NULL AUTO_INCREMENT,
location VARCHAR(50) NOT NULL,
PRIMARY KEY (location_id)
)
"""

insert_data = f"""
INSERT INTO locations (location)
VALUES
  ('USA'),
  ('Great Britain'),
  ('Australia');
"""

query = """
SELECT * FROM locations;
"""

run_update(create_table, db2, 'create')
run_update(insert_data, db2, 'insert')
run_query(query, db2)

,location_id,location
0,1,USA
1,2,Great Britain
2,3,Australia


In [56]:
# ADD column called location_id in members table

create_table = f"""
ALTER TABLE members
ADD COLUMN location_id INT
"""
run_update(create_table, db2, 'alter')
run_query('Select* from members', db2)

,member_id,fname,lname,instrument,band_name,vocals,location_id
0,1,Paul,Bullard,Guitar,Roof Diver,yes,None
1,2,Cameron,McClain,Bass,Roof Diver,yes,None
2,3,Eric,Elliot,Guitar,Roof Diver,no,None
3,4,Rodney,Walton,Drums,Roof Diver,no,None
4,5,John,Lennon,Guitar,Rolling Stones,yes,None
5,6,Paul,McCartney,Drums,The Beatles,no,None
6,7,Ringo,Star,Drums,The Beatles,yes,None
7,8,George,Hamilton,Guitar,The Beatles,yes,None


In [57]:
# Update location_id appropriately
update_location = f"""
UPDATE members
SET location_id = (CASE
 WHEN band_name = 'Roof Diver' then 1
 WHEN band_name = 'The Beatles' then 2
 ELSE 4
 END
)
"""

run_update(update_location, db2, 'update')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals,location_id
0,1,Paul,Bullard,Guitar,Roof Diver,yes,1
1,2,Cameron,McClain,Bass,Roof Diver,yes,1
2,3,Eric,Elliot,Guitar,Roof Diver,no,1
3,4,Rodney,Walton,Drums,Roof Diver,no,1
4,5,John,Lennon,Guitar,Rolling Stones,yes,4
5,6,Paul,McCartney,Drums,The Beatles,no,2
6,7,Ringo,Star,Drums,The Beatles,yes,2
7,8,George,Hamilton,Guitar,The Beatles,yes,2


### INNER JOIN
The inner join is the most common type of join, used when you want to combine all matching values between two tables into a new table, below only the rolling stones location_id doesn't match (it was a 4)

In [59]:
query = """
SELECT m.band_name AS band, l.location, l.location_id
FROM members AS m
INNER JOIN locations AS l
ON m.location_id = l.location_id
"""

run_query(query, db2)

,band,location,location_id
0,Roof Diver,USA,1
1,Roof Diver,USA,1
2,Roof Diver,USA,1
3,Roof Diver,USA,1
4,The Beatles,Great Britain,2
5,The Beatles,Great Britain,2
6,The Beatles,Great Britain,2


In [76]:
# Gets only the distinct values
query = """
SELECT DISTINCT m.band_name AS band, l.location
FROM members AS m
INNER JOIN locations AS l
ON m.location_id = l.location_id

"""

run_query(query, db2)

,band,location
0,Roof Diver,USA
1,The Beatles,Great Britain


### DIAL JOINS BETTER, FINISH WITH BETTER DATA EXAMPLE

## 4 FUNCTIONS
There are a number of functions in SQL, most if not all are aggregate in nature (i.e. SUM(), MAX(), COUNT())

Below I perform some aggregate functions on the movie data Including, AVG(), MAX(), COUNT()

In [83]:
# Get the Average runtime for a movie
query = """
SELECT AVG(runtime)
FROM movies
"""
run_query(query, db1)

,AVG(runtime)
0,100.119


In [87]:
# Get the movie with the longest runtime
query = """
SELECT title, MAX(runtime)
FROM movies
"""
run_query(query, db1)

,title,MAX(runtime)
0,Norm of the North: King Sized Adventure,228


In [96]:
# COUNT the number of movies with runtime > the average runtime of 100.119 minutes
# Get the movie with the longest runtime
query = """
SELECT COUNT(*)
FROM movies
WHERE runtime > 100
"""
run_query(query, db1)

,COUNT(*)
0,1863


## 5. SubQueries (IN)
A Subquery (aka: Inner query, aka: Nested query) is a query within another SQL query and embedded within the WHERE clause. Subqueries are used to return data that will be used in the main query as a condition to further restrict the data to be retrieved.

Subqueries can be used with the SELECT, INSERT, UPDATE, and DELETE statements along with the operators like =, <, >, >=, <=, IN, BETWEEN, etc.

There are a few caveats to subqueries including:

* Subqueries must be enclosed within parentheses
* A subquery can have only one column in the SELECT clause, unless multiple columns are in the main query for the subquery to compare its selected columns
* An ORDER BY command cannot be used in a subquery, although the main query can use an ORDER BY. The GROUP BY command can be used to perform the same function as the ORDER BY in a subquery
* Subqueries that return more than one row can only be used with multiple value operators such as the IN operator.
* The SELECT list cannot include any references to values that evaluate to a BLOB, ARRAY, CLOB, or NCLOB
* A subquery cannot be immediately enclosed in a set function
* The BETWEEN operator cannot be used with a subquery. However, the BETWEEN operator can be used within the subquery

In [103]:
# Normal method for requesting multiple column values
query1 = f"""
SELECT *
FROM `movies`
WHERE (release_date = 2019 OR release_date = 2020)
ORDER BY release_date DESC
LIMIT {limit}
"""

# SubQuery method
query2 = f"""
SELECT *
FROM `movies`
WHERE release_date IN (2017, 2020)
ORDER BY release_date DESC
LIMIT {limit}
"""

run_query(query2)

,id,title,director,release_date,runtime,rating,category
0,81006825,All the Freckles in the World,Yibran Asuad,2020,90,TV-14,Comedies
1,81127902,A Fall from Grace,Tyler Perry,2020,121,TV-MA,Dramas
2,80233408,"Live Twice, Love Once",Maria Ripoll,2020,102,TV-MA,Comedies
3,81060049,Leslie Jones: Time Machine,David Benioff,2020,66,TV-MA,Stand-Up Comedy
4,81214114,Bulletproof 2,Don Michael Paul,2020,97,TV-MA,Action & Adventure


### Subqueries Used as Filters
Subqueries can be used to filter one table from values of another (NOTE BOTH TABLES MUST MATCH ON SAID VALUE)

In [175]:
# In the example below, the movies table is region filtered by id

query1 = """
DESCRIBE region
"""
print('The Region Table Has 2 Fields')
print(run_query(query1))


query2 = f"""
SELECT DISTINCT region_code
FROM `region`
ORDER BY region_code ASC
"""
print()
print()
print('There are 10 unique region_code values:')
print(run_query(query2).region_code.unique())


query3 = f"""
SELECT * FROM movies
WHERE id IN (SELECT id FROM region WHERE region_code = 4)
AND rating = 'R'
ORDER BY runtime ASC
"""
run_query(query3)

The Region Table Has 2 Fields
         Field    Type Null Key Default Extra
0           id  b'int'  YES        None      
1  region_code  b'int'  YES        None      


There are 10 unique region_code values:
[ 1  2  3  4  5  6  7  8  9 10]


,id,title,director,release_date,runtime,rating,category
0,60024976,Jerry Seinfeld: Comedian,Christian Charles,2002,81,R,Documentaries
1,70211216,Kevin Hart: Laugh at My Pain,Leslie Small,2011,89,R,Stand-Up Comedy
2,80037280,The Witch,Robert Eggers,2015,92,R,Horror Movies
3,80017286,While We're Young,Noah Baumbach,2015,97,R,Comedies
4,80240972,The Kindergarten Teacher,Sara Colangelo,2018,97,R,Dramas
5,80028357,"Love, Rosie",Christian Ditter,2014,103,R,Comedies
6,70024088,Brick,Rian Johnson,2005,110,R,Independent Movies
7,485218,Executive Decision,Stuart Baird,1996,133,R,Action & Adventure
8,80175694,Mudbound,Dee Rees,2017,135,R,Dramas


### SubQuries Within SubQueries
Subqueries can be chained, but this can get complicated quickly and is not recommended